<a href="https://colab.research.google.com/github/dcatrib/tesePython/blob/main/tese_PyMuPDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install PyMuPDF requests selenium chromedriver-autoinstaller pandas nltk gensim numpy

!apt-get update
!apt-get install -y wget
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt-get install -y ./google-chrome-stable_current_amd64.deb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,160 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,325 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,122 kB]
Ign:9 https://r2u.stat.illino

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
import chromedriver_autoinstaller
import requests
import fitz
import os
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from google.colab import files
import numpy as np
from gensim.models import KeyedVectors, Word2Vec
import gensim
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.ar_model import AutoReg


In [6]:
# Configurando o WebDriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Instala automaticamente a versão correta do ChromeDriver
chromedriver_autoinstaller.install()



'/usr/local/lib/python3.10/dist-packages/chromedriver_autoinstaller/129/chromedriver'

In [39]:
# URL para scrape
url = "https://www.bcb.gov.br/en/publications/copomminutes/cronologicos"

# Criar uma nova instância do Chrome driver
driver = webdriver.Chrome(options=chrome_options)
driver.get(url)

# Esperar o seletor específico estar presente
WebDriverWait(driver, 15).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'body > app-root > app-root > main > dynamic-comp > div > div > bcb-publicacao > div > div > bcb-ultimaspublicacoes > div'))
)

# Definir o seletor
selector = 'body > app-root > app-root > main > dynamic-comp > div > div > bcb-publicacao > div > div > bcb-ultimaspublicacoes > div'

# Lista para armazenar hrefs, textos e links de download
hrefs_and_texts = []


In [40]:
# Função para recuperar links e botões de download
def retrieve_links():
    try:
        elements = driver.find_elements(By.CSS_SELECTOR, selector)
        link_count = 0  # Inicializa um contador para os links
        for element in elements:
            links = element.find_elements(By.TAG_NAME, 'a')
            for link in links:
                if link_count >= 183:  # Limita a 50 links
                    return  # Sai da função se 50 links foram processados
                href = link.get_attribute('href')
                text = link.text
                if href and text and text[0].isdigit():
                    driver.get(href)
                    # Esperar o botão de download
                    try:
                        download_button = WebDriverWait(driver, 45).until(
                            EC.presence_of_element_located((By.CSS_SELECTOR, '#publicacao > div.col-lg-9.d-flex.flex-column > div > div > div > div.d-flex.flex-column-reverse.flex-sm-row.justify-content-sm-between.align-items-sm-center > div.d-flex.align-items-center.mb-3.mb-sm-0.d-print-none > download > div > div > a'))
                        )
                        download_href = download_button.get_attribute('href')

                        # Fazer download do PDF
                        pdf_response = requests.get(download_href)
                        pdf_filename = f"{text}.pdf"
                        with open(pdf_filename, 'wb') as pdf_file:
                            pdf_file.write(pdf_response.content)

                        # Extrair texto do PDF usando PyMuPDF
                        pdf_text = ''
                        with fitz.open(pdf_filename) as pdf_file:
                            for page in pdf_file:
                                page_text = page.get_text()
                                pdf_text += page_text + '\n'

                        # Limpeza do texto
                        pdf_text = pdf_text.replace('\n', ' ')  # Remove quebras de linha
                        pdf_text = re.sub(r'\s+', ' ', pdf_text)  # Substitui múltiplos espaços por um único
                        pdf_text = pdf_text.strip()  # Remove espaços no início e no fim

                        # Print da primeira linha extraída
                        first_line = pdf_text.split('.')[0]  # Considera a primeira frase
                        #print(f'Primeira linha extraída: {first_line.strip()}')
                        print(link_count)
                    except (NoSuchElementException, TimeoutException):
                        download_href = None
                        pdf_text = None

                    hrefs_and_texts.append((href, text, download_href, pdf_text))
                    link_count += 1  # Incrementa o contador
                    driver.back()
                    WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, selector)))
    except StaleElementReferenceException:
        retrieve_links()

# Iniciar a recuperação de links
retrieve_links()

# Fechar o driver
driver.quit()


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182


Minhas stopwords e expressoes

In [41]:
# Minhas stopwords
my_stopwords = {
    'January', 'February', 'March', 'April', 'May', 'June', 'July',
    'August', 'September', 'October', 'November', 'December','january', 'february', 'march', 'april', 'may', 'june', 'July',
    'august', 'september', 'october', 'november', 'december',
    'minutes', 'bcb', 'th', 'copom', 'bcbgovbr', 'brasilia', 'pm','roberto'
}

# Array de expressões a serem removidas
expressions_to_remove = [
    'bcb.gov.br Minutes of the Meeting of the Monetary Policy Committee — Copom',
]

fazer o preprocessamento e criar o df

In [42]:
# Baixar as stopwords se ainda não tiver feito
nltk.download('stopwords')

# Criar um DataFrame com os links e textos extraídos
df = pd.DataFrame(hrefs_and_texts, columns=['Link', 'Texto', 'Link de Download', 'Texto do PDF'])

# Carregar stopwords em inglês
stop_words = set(stopwords.words('english'))
stop_words.update(my_stopwords)

# Função para preprocessar o texto
def preprocess_text(text):
    if text:
        # Remover números, pontuação e caracteres especiais
        text = re.sub(r'[0-9]+', '', text)  # Remove números
        # Remover pontuação, exceto parênteses
        text = re.sub(r'[^\w\s()]+', '', text)  # Mantém letras, números, espaços e parênteses
        text = re.sub(r'\s+', ' ', text)  # Substituir múltiplos espaços por um único espaço
        text = text.strip().lower()  # Retorna texto em minúsculas

        # Remover expressões específicas
        for expr in expressions_to_remove:
            text = text.replace(expr, '')

        # Remover stopwords
        text = ' '.join([word for word in text.split() if word not in stop_words])
        return text
    return ''

# Criar a nova coluna 'Texto Processado'
df['Texto Processado'] = df['Texto do PDF'].apply(preprocess_text)

# Imprimir o DataFrame
print(df)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


                                                  Link  \
0    https://www.bcb.gov.br/en/publications/copommi...   
1    https://www.bcb.gov.br/en/publications/copommi...   
2    https://www.bcb.gov.br/en/publications/copommi...   
3    https://www.bcb.gov.br/en/publications/copommi...   
4    https://www.bcb.gov.br/en/publications/copommi...   
..                                                 ...   
178  https://www.bcb.gov.br/en/publications/copommi...   
179  https://www.bcb.gov.br/en/publications/copommi...   
180  https://www.bcb.gov.br/en/publications/copommi...   
181  https://www.bcb.gov.br/en/publications/copommi...   
182  https://www.bcb.gov.br/en/publications/copommi...   

                                     Texto  \
0    265th Meeting - September 17-18, 2024   
1         264th Meeting - July 30-31, 2024   
2         263rd Meeting - June 18-19, 2024   
3            262nd Meeting - May 7-8, 2024   
4        261st Meeting - March 19-20, 2024   
..                         

In [43]:
# Função para extrair ano e mês
def extract_year_month(text):
    # Caso 1: Formato com dia e mês
    match_full = re.search(r'(\w+)\s+\d{1,2}[- ]\d{1,2},\s*(\d{4})', text)
    # Caso 2: Formato apenas com mês e ano
    match_month_year = re.search(r'(\w+)\s+(\d{4})', text)
    # Caso 3: Formato com intervalo de datas
    match_interval = re.search(r'(\w+)\s+\d{1,2}\s*-\s*(\w+)\s+\d{1,2},\s*(\d{4})', text)

    if match_full:
        month_str = match_full.group(1)
        year = match_full.group(2)
    elif match_month_year:
        month_str = match_month_year.group(1)
        year = match_month_year.group(2)
    elif match_interval:
        month_str = match_interval.group(2)  # Captura o mês da última data
        year = match_interval.group(3)
    else:
        return None

    # Converter mês para número
    month_num = pd.to_datetime(month_str, format='%B').month
    return f"{year}-{month_num:02d}"  # Formatar como YYYY-MM

# Criar nova coluna com ano e mês
df['Ano_Mes'] = df['Texto'].apply(extract_year_month)

# Exibir o DataFrame resultante
print(df)

                                                  Link  \
0    https://www.bcb.gov.br/en/publications/copommi...   
1    https://www.bcb.gov.br/en/publications/copommi...   
2    https://www.bcb.gov.br/en/publications/copommi...   
3    https://www.bcb.gov.br/en/publications/copommi...   
4    https://www.bcb.gov.br/en/publications/copommi...   
..                                                 ...   
178  https://www.bcb.gov.br/en/publications/copommi...   
179  https://www.bcb.gov.br/en/publications/copommi...   
180  https://www.bcb.gov.br/en/publications/copommi...   
181  https://www.bcb.gov.br/en/publications/copommi...   
182  https://www.bcb.gov.br/en/publications/copommi...   

                                     Texto  \
0    265th Meeting - September 17-18, 2024   
1         264th Meeting - July 30-31, 2024   
2         263rd Meeting - June 18-19, 2024   
3            262nd Meeting - May 7-8, 2024   
4        261st Meeting - March 19-20, 2024   
..                         

In [44]:
# Definir as expressões que você deseja encontrar
expressions = [
    r'(?i)(A\)\s*update (?:e conomic|economic) outlook (?:c opoms|copoms) (?:reference|baseline|)\s*scenario)(.*)',
    r'(?i)(Recent Economic Developments)(.*)',
    r'(?i)(Recent Evolution Inflation)(.*)',
]

# Função para extrair texto a partir de várias expressões
def extract_from_expressions(text):
    for pattern in expressions:
        match = re.search(pattern, text)
        if match:
            # Retorna o texto a partir da expressão encontrada até o final
            return text[match.start():]  # A partir do início da expressão até o final do texto
    return text  # Retorna todo o texto se nenhuma expressão for encontrada

df['Texto Processado'] = df['Texto Processado'].apply(extract_from_expressions)


# Definir os títulos das seções, permitindo variações com ou sem espaço
section_titles = [
    r'(?i)A\)\s*update (?:e conomic|economic) outlook (?:c opoms|copoms) (?:reference|baseline|)\s*scenario.*?(?=B\)\s*risks around|B\)\s*scenarios risk analysis|$)',
    r'(?i)(B\)\s*risks around|B\)\s*scenarios risk analysis).*?(?=C\)\s*discussion conduct monetary policy|$)',
    r'(?i)C\)\s*discussion conduct monetary policy.*?(?=D\)\s*monetary policy decision|$)',
    r'(?i)D\)\s*monetary policy decision.*?(?=Footnotes|$)',
    r'(?i)Footnotes.*'  # Captura tudo até o final do texto
]


# Criar variáveis para cada seção no DataFrame
for title in section_titles:
    # Usar str.extract para capturar as seções
    section_name = re.search(r'A|B|C|D|Footnotes', title).group(0)  # Captura A, B, C, D ou Footnotes
    df[section_name] = df['Texto Processado'].str.extract(f'({title})')[0]

print (df[['Texto Processado']])
# Exibir o DataFrame resultante
#print(df[['A', 'B', 'C', 'D', 'Footnotes']])

                                      Texto Processado
0    a) update economic outlook copoms scenario glo...
1    a) update economic outlook copoms scenario glo...
2    a) update economic outlook copoms scenario glo...
3    a) update economic outlook copoms scenario glo...
4    a) update economic outlook copoms scenario glo...
..                                                 ...
178  recent evolution inflation inflation rates jul...
179  recent evolution inflation inflation continued...
180  recent evolution inflation measures inflation ...
181  recent evolution inflation measures inflation ...
182  recent evolution inflation consumer price infl...

[183 rows x 1 columns]


Carregar a base dos pibs trimestrais

In [45]:
# https://www.ibge.gov.br/estatisticas/economicas/contas-nacionais/9300-contas-nacionais-trimestrais.html?=&t=series-historicas&utm_source=landing&utm_medium=explica&utm_campaign=pib#evolucao-taxa

# Fazendo upload do arquivo CSV
uploaded = files.upload()

# Carregar o arquivo CSV usando o delimitador correto
df_trimestres = pd.read_csv('pib_trimestral.csv', delimiter=';', header=1)  # O cabeçalho correto

# Excluir a primeira coluna se necessário (baseado no que você descreveu)
df_trimestres = df_trimestres.iloc[:1, 1:]  # Remove a primeira coluna


# Verifique o DataFrame após as alterações
print("DataFrame após a leitura:")
print(df_trimestres.head())

# Agora, precisamos reorganizar os dados
# Vamos usar melt para transformar os dados
df_trimestres_melted = df_trimestres.melt(var_name='Trimestre', value_name='Valor')

# Verifique o DataFrame derretido
print("DataFrame derretido:")
print(df_trimestres_melted.head())

# Função para gerar mapeamento de trimestres
def gerar_mapeamento_trimestres(inicio_ano, fim_ano):
    mapeamento = {}
    for ano in range(inicio_ano, fim_ano + 1):
        for trimestre in range(1, 5):
            mes = trimestre * 3  # 1: Março, 2: Junho, 3: Setembro, 4: Dezembro
            ano_mes = f"{ano}-{mes:02d}"
            mapeamento[f"{trimestre}º trimestre {ano}"] = ano_mes
    return mapeamento

# Gerar mapeamento de 1996 a 2024
mapeamento_trimestre = gerar_mapeamento_trimestres(1996, 2024)
df_trimestres_melted['Ano_Mes'] = df_trimestres_melted['Trimestre'].map(mapeamento_trimestre)

print("DataFrame com o mapeamento:")
print(df_trimestres_melted[['Trimestre', 'Ano_Mes']].head(20))  # Exibir as primeiras 20 linhas

# Verificar se há valores ausentes na coluna Ano_Mes
valores_ausentes = df_trimestres_melted['Ano_Mes'].isna().sum()
print(f"Número de valores ausentes em 'Ano_Mes': {valores_ausentes}")

# Exibir trimestres e seus respectivos Ano_Mes
for trimestre in df_trimestres_melted['Trimestre'].unique():
    print(f"{trimestre}: {df_trimestres_melted[df_trimestres_melted['Trimestre'] == trimestre]['Ano_Mes'].values}")


# Verifique o resultado do mapeamento
print("Trimestres únicos:")
print(df_trimestres_melted['Trimestre'].unique())
print(df_trimestres_melted[['Ano_Mes', 'Valor']])


# Agora, mescle os dois dataframes (supondo que o df original esteja carregado em df)
df_final = pd.merge(df, df_trimestres_melted[['Ano_Mes', 'Valor']], on='Ano_Mes', how='outer')
#df_final = df_final.dropna(subset=['Texto Processado'])

#drop de 2003-03 para tras
# Filtra para manter apenas os registros a partir de 2003-03
df_final = df_final[df_final['Ano_Mes'] >= '2003-04']

# Verifique o resultado final
print(df_final.head())

Saving pib_trimestral.csv to pib_trimestral.csv
DataFrame após a leitura:
   1º trimestre 1996  2º trimestre 1996  3º trimestre 1996  4º trimestre 1996  \
0                2.5                2.1                2.3                2.2   

   1º trimestre 1997  2º trimestre 1997  3º trimestre 1997  4º trimestre 1997  \
0                3.2                4.0                3.0                3.4   

   1º trimestre 1998  2º trimestre 1998  ...  1º trimestre 2022  \
0                2.8                2.0  ...                4.7   

   2º trimestre 2022  3º trimestre 2022  4º trimestre 2022  1º trimestre 2023  \
0                2.7                2.7                3.0                3.7   

   2º trimestre 2023  3º trimestre 2023  4º trimestre 2023  1º trimestre 2024  \
0                3.7                3.1                2.9                2.5   

   2º trimestre 2024  
0                2.5  

[1 rows x 114 columns]
DataFrame derretido:
           Trimestre  Valor
0  1º trimestre 1996

Treinando o modelo Word2vec. Utilizando o Texto Processado como corpus (A PARTIR DAQUI COM O XLSX)

In [47]:
# Carregar o CSV (Para nao precisar rodar tudo novamente)
import requests
import os
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from google.colab import files
import numpy as np
from gensim.models import KeyedVectors, Word2Vec
import gensim
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.ar_model import AutoReg

#-------------------------------------------------------------
#excel

# Faça o upload do arquivo
#uploaded = files.upload()

# Carregando o arquivo Excel corretamente
#df_final = pd.read_excel('corpus.xlsx')  # Corrigido para corpus.xlsx

# Extraindo a quarta coluna como corpus
#corpus = df.iloc[:, 4].tolist()  # Altere o índice se necessário

#---------------------------------------------------------------------
#DF
# Extraindo a quarta coluna como corpus
corpus = df_final['Texto Processado'].tolist()  # Altere o índice se necessário


#---------------------------------------------------------------------
corpus = [text for text in corpus if isinstance(text, str) and text.strip() != ""]

# Pré-processamento do texto (divisão em palavras)
# Aqui estamos usando um simples split, mas você pode querer fazer mais pré-processamento
corpus_tokenized = [text.split() for text in corpus]

# Word2Vec (cbow)
print("Treinando Word2Vec...")
word2vec_model = gensim.models.Word2Vec(corpus_tokenized, vector_size=100, window=5, min_count=1, workers=4)

Treinando Word2Vec...


Criando o vetor medio dos vetores medios das atas referente ao trimestre

In [48]:
# Função para obter o vetor médio
def get_average_vector(text):
    if isinstance(text, str):  # Verifica se o texto é uma string
        words = text.split()
        word_vectors = [word2vec_model.wv[word] for word in words if word in word2vec_model.wv]
        if word_vectors:
            return np.mean(word_vectors, axis=0)
    return np.zeros(word2vec_model.vector_size)  # Retorna vetor zero se não houver palavras válidas ou se o texto não for string

# Criar a nova coluna 'Texto Processado'
df_final['Vetor Médio'] = df_final['Texto Processado'].apply(get_average_vector)

# Inicializa uma lista para os vetores médios do trimestre
vetores_medio_trimeste = []
previous_vectors = []  # Lista para acumular vetores das linhas com valor missing

# Para cada linha no DataFrame
for i in range(len(df_final)):
    current_vector = df_final['Vetor Médio'].iloc[i]

    # Se o valor não for missing
    if not np.isnan(df_final['Valor'].iloc[i]):
        has_zero_vector = np.array_equal(current_vector, np.zeros(word2vec_model.vector_size))

        # Se houver vetores acumulados, calcula a média
        if previous_vectors:
            # Filtra apenas vetores válidos para o cálculo da média
            valid_vectors = [v for v in previous_vectors if not np.array_equal(v, np.zeros(word2vec_model.vector_size))]
            if valid_vectors:
                # Calcula a média entre o vetor acumulado e o vetor atual
                vetor_medio_final = np.mean([np.mean(valid_vectors, axis=0), current_vector], axis=0)
                if has_zero_vector:
                    vetor_medio_final *= 2  # Multiplica por 2 se houver pelo menos um vetor inválido
                vetores_medio_trimeste.append(vetor_medio_final)
            else:
                vetores_medio_trimeste.append(current_vector * 2)  # Se não houver vetores válidos, usa o vetor atual
        else:
            vetores_medio_trimeste.append(current_vector)

        # Limpa a lista acumulada para o próximo conjunto
        previous_vectors = []
    else:
        # Se o valor for missing, acumula o vetor médio
        if not np.array_equal(current_vector, np.zeros(word2vec_model.vector_size)):
            previous_vectors.append(current_vector)
        vetores_medio_trimeste.append(None)  # Adiciona None para os missing

# Adiciona a nova coluna ao DataFrame
df_final['vetor_medio_trimestre'] = vetores_medio_trimeste

# Verifica o resultado
#print(df_final[['Texto Processado', 'Valor', 'Vetor Médio', 'vetor_medio_trimestre']].head(20))

Previsao

In [69]:
# Descartar as observações onde a coluna 'Valor' é missing (NaN)
df_final_cleaned = df_final.dropna(subset=['Valor'])

# Verificar o resultado
print(df_final_cleaned['vetor_medio_trimestre'])  # Exibe as primeiras linhas do DataFrame limpo

# X
X = np.array(df_final_cleaned['vetor_medio_trimestre'].tolist())

# Y: Valores a serem previstos
y = df_final_cleaned['Valor'].values

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Treinando o modelo de regressão
model = LinearRegression()

# Realiza a validação cruzada com 5 dobras (folds)
cv_scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')

# Converte os scores para valores positivos
mse_scores = -cv_scores

# Calcula a média e o desvio padrão do MSE
mean_mse = np.mean(mse_scores)
std_mse = np.std(mse_scores)

# Treinando o modelo com todos os dados
model.fit(X, y)

# Fazer previsões (opcional, se você quiser comparar com previsões em um conjunto de teste)
#predictions = model.predict(X)

# Fazer previsões no conjunto de teste
predictions_test = model.predict(X_test)

# Calcular o MSE para as previsões no conjunto de teste
mse_linear = mean_squared_error(y_test, predictions_test)

# Criar um DataFrame para comparar previsões e valores reais
results_df = pd.DataFrame({
    'Valor Real': y_test,
    'Previsão': predictions_test
})

# Ajustar um modelo autoregressivo (AR) usando apenas y
model_ar2 = AutoReg(y, lags=2)  # Use o número de lags apropriado
model_fit_ar2 = model_ar2.fit()
print(model_fit_ar2.summary())

# Fazer previsões
predictions_ar2 = model_fit_ar2.predict(start=len(y_train), end=len(y_train) + len(y_test) - 1)
print(predictions_ar2)

# Calcular o erro médio quadrático (MSE) para AR
mse_ar2 = mean_squared_error(y_test, predictions_ar2)

results_df['Previsão AR(2)'] = predictions_ar2  # Adiciona a nova coluna com previsões do AR(2)

# Verificar os resultados
print(results_df)

# Imprimir todos os resultados para comparação
print(f'Mean MSE (Linear Regression - CV): {mean_mse}')
print(f'Standard Deviation of MSE (Linear Regression - CV): {std_mse}')
print(f'Mean Squared Error (Linear Regression - Test Set): {mse_linear}')
print(f'Mean Squared Error AR(2): {mse_ar2}')

31     [-0.36054844, 0.10639507, 0.28305176, 0.337040...
34     [-0.324018, 0.118333995, 0.3041522, 0.3236421,...
37     [-0.27688137, 0.12061549, 0.27072445, 0.297198...
40     [-0.34773308, 0.14291504, 0.27324387, 0.356159...
43     [-0.3437688, 0.17840719, 0.3018998, 0.33651412...
                             ...                        
218    [-0.59499896, 0.21484083, -0.01446094, 0.18904...
220    [-0.58919513, 0.2061033, -0.042458683, 0.21070...
222    [-0.6102571, 0.24269715, -0.025317485, 0.15900...
224    [-0.64068836, 0.22806332, -0.05927429, 0.15277...
226    [-0.60022545, 0.2745545, -0.021278743, 0.19165...
Name: vetor_medio_trimestre, Length: 85, dtype: object
                            AutoReg Model Results                             
Dep. Variable:                      y   No. Observations:                   85
Model:                     AutoReg(2)   Log Likelihood                -106.716
Method:               Conditional MLE   S.D. of innovations              0.875
Da

In [50]:
#exportando para um arquivo csv
#print(df_final.head())
#output_csv = 'dados_extraidos.csv'  # Nome do arquivo CSV
#df_final.to_csv(output_csv, index=False)

# Exportando para Excel
output_excel = 'dados_extraidos.xlsx'
df_final.to_excel(output_excel, index=False)

# Download no Google Colab
files.download(output_excel)

# Se você estiver usando Google Colab, faça o download do arquivo CSV
#try:
    #from google.colab import files
    #files.download(output_csv)
#except ImportError:
    #print("Não é um ambiente Colab. O arquivo está salvo como:", output_csv)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

--------------------------------------------------------------------------------

Sintonia fina de base pre treinada

In [ ]:
import numpy as np
from gensim.models import KeyedVectors, Word2Vec
import os
from google.colab import drive

drive.mount('/content/drive')

# Defina o caminho para os embeddings
embeddings_path = '/content/drive/My Drive/Caen/TESE/word2vec-google-news-300.model.vectors.npy'

# Carregar os vetores
vectors = np.load(embeddings_path)

# Tokenizar cada frase do seu DataFrame
corpus = df['Texto Processado'].tolist()
tokenized_corpus = [sentence.split() for sentence in corpus]  # Tokenização simples

# Criar um novo modelo Word2Vec
word2vec_model = Word2Vec(vector_size=300, window=5, min_count=1, sg=1)

# Construir o vocabulário do corpus
word2vec_model.build_vocab(tokenized_corpus)

# Inicializar os vetores do novo modelo com os vetores pré-treinados
for word in word2vec_model.wv.index_to_key:
    if word in vectors:  # Verifique se a palavra existe nos vetores
        word2vec_model.wv[word] = vectors[vectors.index(word)]  # Atribua o vetor pré-treinado

# Treinar o modelo no seu conjunto de dados
word2vec_model.train(tokenized_corpus, total_examples=len(tokenized_corpus), epochs=word2vec_model.epochs)

# Salvar o novo modelo
word2vec_model.save("/content/drive/My Drive/Caen/TESE/meu_modelo_fine_tuned_word2vec.model")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
